In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os

print(os.getcwd())
# Specify location of data
os.chdir("../data/recommender-dataset")
print(os.getcwd())

2022-07-14 12:50:18.266083: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-07-14 12:50:18.266107: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/cory/PycharmProjects/bachelor_2022/artifact/data_fetch
/home/cory/PycharmProjects/bachelor_2022/artifact/data/recommender-dataset


In [2]:
# Specify dataset
ds_name = '100k-ratings'
# Load dataset
ds = tfds.load(name=f'movielens/{ds_name}', data_dir=os.getcwd(), download=False, split="all")

2022-07-14 12:50:33.332856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-14 12:50:33.333115: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-07-14 12:50:33.333187: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-07-14 12:50:33.333244: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-07-14 12:50:33.333298: W tensorf

In [3]:
df = tfds.as_dataframe(ds)
median_timestamp = df['timestamp'].median()

In [4]:
# Split data in half by timestamp
df_1 = df.loc[df['timestamp'] <= median_timestamp]
df_2 = df.loc[df['timestamp'] > median_timestamp]

# Check split
print(df_1.shape)
print(df_2.shape)
print(type(df_1))

df_1.columns

(50002, 12)
(49998, 12)
<class 'pandas.core.frame.DataFrame'>


Index(['bucketized_user_age', 'movie_genres', 'movie_id', 'movie_title',
       'raw_user_age', 'timestamp', 'user_gender', 'user_id',
       'user_occupation_label', 'user_occupation_text', 'user_rating',
       'user_zip_code'],
      dtype='object')

In [5]:
# Convert split Dataframes back to tf Datasets

ds_1 = (
        tf.data.Dataset.from_tensor_slices(
            (
                tf.cast(df_1['bucketized_user_age'].values, tf.float32),
                #tf.cast(df_1['movie_genres'].values, tf.int64),
                tf.cast(df_1['movie_id'].values, tf.string),
                tf.cast(df_1['movie_title'].values, tf.string),
                tf.cast(df_1['raw_user_age'].values, tf.float32),
                tf.cast(df_1['timestamp'].values, tf.int64),
                tf.cast(df_1['user_gender'].values, tf.bool),
                tf.cast(df_1['user_id'].values, tf.string),
                tf.cast(df_1['user_occupation_label'].values, tf.int64),
                tf.cast(df_1['user_occupation_text'].values, tf.string),
                tf.cast(df_1['user_rating'].values, tf.float32),
                tf.cast(df_1['user_zip_code'].values, tf.string),

            )
        )
    )

ds_2 = (
        tf.data.Dataset.from_tensor_slices(
            (
                tf.cast(df_2['bucketized_user_age'].values, tf.float32),
                #tf.cast(df_2['movie_genres'].values, tf.int64),
                tf.cast(df_2['movie_id'].values, tf.string),
                tf.cast(df_2['movie_title'].values, tf.string),
                tf.cast(df_2['raw_user_age'].values, tf.float32),
                tf.cast(df_2['timestamp'].values, tf.int64),
                tf.cast(df_2['user_gender'].values, tf.bool),
                tf.cast(df_2['user_id'].values, tf.string),
                tf.cast(df_2['user_occupation_label'].values, tf.int64),
                tf.cast(df_2['user_occupation_text'].values, tf.string),
                tf.cast(df_2['user_rating'].values, tf.float32),
                tf.cast(df_2['user_zip_code'].values, tf.string),

            )
        )
    )


# Check types
print(type(df_1))
print(type(ds_1))

<class 'pandas.core.frame.DataFrame'>
<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>


In [43]:
# Put datasets in a list to execute write process in a loop
ds_list = [ds_1, ds_2]

for i in range(len(ds_list)):
    # Define & create dir
    data_path = f'movielens/{ds_name}-split_{i+1}'
    if os.path.exists(data_path) is False:
        os.mkdir(data_path)
    # Serialize Dataset by converting each tensor and save it in a separate tfrecord file
    for j, _ in enumerate(ds_list[i].element_spec):
        serialized_tensor = ds_list[i].map(lambda *args: args[j]).map(tf.io.serialize_tensor)
        writer = tf.data.experimental.TFRecordWriter(data_path + f'/movielens-{j}.tfrecord')
        writer.write(serialized_tensor)

In [6]:
data_path = f'movielens/test'
tf.data.experimental.save(ds_1, data_path)